In [5]:
# 1. Đảm bảo bạn đã cài đặt tất cả thư viện:
#    pip install langchain openai

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from src.llm import gemini_llm       # LLM client của bạn, đã config API key ở src/llm.py

# 2. Đọc M-Schema từ file
with open('src/schema/m_schema.txt', 'r', encoding='utf-8') as f:
    schema = f.read().strip()

# 3. Tạo PromptTemplate
prompt = PromptTemplate(
    input_variables=["schema", "question"],
    template="""
You are a SQL expert.
Given the table schema:
{schema}

Translate the following natural language question into SQL:
Question: {question}
SQL:
"""
)

# 4. Khởi tạo chain
sql_generator = LLMChain(
    llm=gemini_llm,
    prompt=prompt,
    verbose=True          
)

# 5. Chạy thử
question = "Doanh thu dịch vụ Thu hộ Dịch vụ công quý 2 năm 2024"
sql = sql_generator.run(schema=schema, question=question)

print("Generated SQL:")
print(sql.strip())


C:\Users\Admin\AppData\Local\Temp\ipykernel_23232\1138578974.py:27: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  sql_generator = LLMChain(
C:\Users\Admin\AppData\Local\Temp\ipykernel_23232\1138578974.py:35: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  sql = sql_generator.run(schema=schema, question=question)




> Entering new LLMChain chain...
Prompt after formatting:

You are a SQL expert.
Given the table schema:
# Table: CHATBOT_KTDL_TARGETS_REVENUE
[
  (SERVICE_PK:VARCHAR(255), PK,   "mã chỉ tiêu",                                    Examples: ["CT_VDS_DT_DV_TH_PUBLIC_SERVICES", "CT_VDS_DT_TOTAL"]),
  (PARTITION_DATE:INT,      ,     "ngày lấy dữ liệu (yyyymmdd)",                    Examples: ["20241218"]),
  (DAY_VALUE:FLOAT,         ,     "giá trị của ngày (thực hiện ngày)",              Examples: ["12345.67"]),
  (MONTH_VALUE:FLOAT,       ,     "lũy kế trong tháng tính từ đầu tháng đến ngày", Examples: ["67890.12"]),
  (QUARTER_VALUE:FLOAT,     ,     "lũy kế trong quý tính từ đầu quý đến ngày",     Examples: ["23456.78"]),
  (YEAR_VALUE:FLOAT,        ,     "lũy kế trong năm tính từ đầu năm đến ngày",     Examples: ["34567.89"]),
  (SPD_PERCENT:FLOAT,       ,     "tỷ lệ chênh lệch ngày giữa hôm nay và hôm qua",  Examples: ["2.0"]),
  (SPM_DELTA:FLOAT,         ,     "chênh lệch lũy kế thá

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from src.llm import gemini_llm

with open("src/schema/service_pk.txt","r", encoding="utf-8") as f:
    mapping = f.read().strip()
with open("src/schema/m_schema.txt", "r", encoding="utf-8") as f:
    m_schema = f.read().strip()


pk_prompt = PromptTemplate(
    input_variables=["mapping", "question"],
    template="""
Dưới đây là danh sách mã chỉ tiêu và mô tả chi tiết:

{mapping}

Hỏi: {question}

Hãy chọn một mã chỉ tiêu (Service_pk) phù hợp nhất với câu hỏi trên:
Answer with only the Service_pk (ví dụ: CT_VDS_DT_DV_TH_PUBLIC_SERVICES).
"""
)
pk_chain = LLMChain(llm=gemini_llm, prompt=pk_prompt, output_key="service_pk", verbose=True)





sql_prompt = PromptTemplate(
    input_variables=["m_schema", "service_pk", "question"],
    template="""
You are a SQL expert.
Given the table schema (M-Schema):
{m_schema}

The user asked: "{question}"

The target primary key in this table is: {service_pk}.

Translate the question into a SQL query on table CHATBOT_KTDL_TARGETS_REVENUE,
filtering by SERVICE_PK = '{service_pk}',
and selecting the columns needed to answer the question. 
Answer with the complete SQL only.
"""


)
sql_chain = LLMChain(llm=gemini_llm, prompt=sql_prompt, output_key="sql", verbose=True)


overall_chain = SequentialChain(
    chains=[pk_chain, sql_chain],
    input_variables=["mapping", "m_schema", "question"],
    output_variables=["service_pk", "sql"],
    verbose=True,
)


question = "Doanh thu dịch vụ Thu hộ Dịch vụ công quý 2 năm 2024"
result = overall_chain({"question": question, "mapping" : mapping, "m_schema" : m_schema})

print("Detected Service_pk:", result["service_pk"])
print("Generated SQL:\n", result["sql"])




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Dưới đây là danh sách mã chỉ tiêu và mô tả chi tiết:

DANH SÁCH MÃ CHỈ TIÊU VÀ MÔ TẢ CHI TIẾT

- CT_VDS_DT_TOTAL  
  • Mã chỉ tiêu CT_VDS_DT_TOTAL được định nghĩa là “Tổng doanh thu”  
  • Tổng doanh thu của Tổng Công ty được cấu thành từ 6 nhóm dịch vụ: SMS banking, Chuyển tiền, Thanh toán số, Tài chính số, Chấm điểm khách hàng và Các dịch vụ khác.

- CT_VDS_DT_DV_TOTAL  
  • Mã chỉ tiêu CT_VDS_DT_DV_TOTAL được định nghĩa là “Doanh thu dịch vụ”  
  • Là doanh thu cung cấp dịch vụ tới khách hàng, bao gồm doanh thu sau chia sẻ trong hợp tác và phần chênh lệch được hưởng (phí hoa hồng, phí đại lý, chênh lệch giá bán - giá nhập) trong hoạt động cung cấp thiết bị, hàng hóa, bán hàng dịch vụ viễn thông; không bao gồm doanh thu hoạt động tài chính và thu nhập khác.

- CT_VDS_DT_SMS_BANKING  
  • Mã chỉ tiêu CT_VDS_DT_SMS_BANKING được định nghĩa là “Doanh thu dịch vụ SMS Banking”  
  • Doanh

: 